<a href="https://colab.research.google.com/github/h4mm4d/Amazon_Vine_Analysis/blob/main/PaidReviewAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-27 23:09:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.88MB/s    in 0.2s    

2022-02-27 23:09:13 (4.88 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# load Amazon data

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = df.dropna()

In [8]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R215C9BDXTDQOW|          4|            0|          0|   N|                Y|
|R1DTPUV1J57YHA|          5|            0|          0|   N|                Y|
| RFAZK5EWKJWOU|          5|            0|          0|   N|                Y|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|                Y|
|R14GRNANKO2Y2J|          5|            0|          0|   N|                Y|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|                Y|
|R1GI9UW5KJ671O|          5|            0|          0|   N|                Y|
|R2H5CEJN863M86|          5|            0|          1|   N|                Y|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|                Y|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|     

In [9]:
#more than 20 votes df
filtered_vine_df = vine_df.filter("total_votes>=20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [10]:
#more than half is helpful reviews votes 
helpful_vine_df = filtered_vine_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [13]:
# paid helpful reviews
paid_helpful_vine_df = helpful_vine_df.filter("vine == 'Y'")
paid_helpful_vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R29V4UDSS053D8|          3|           33|         38|   Y|                N|
|R1I2D068WC37PA|          3|           32|         37|   Y|                N|
|R13W2U74F67QED|          5|           62|         76|   Y|                N|
|R2QI37XFOBKUGD|          4|           24|         33|   Y|                N|
|R19F60BB2DNCKN|          5|          170|        174|   Y|                N|
|R2PYNEWYYQDWHI|          5|           24|         27|   Y|                N|
|R1UEWHCPJL2XJB|          5|           70|         72|   Y|                N|
| RXTMMEBRYZB53|          5|           21|         24|   Y|                N|
|R1PXVYIYMYGUZL|          5|           21|         26|   Y|                N|
| RSFMJJXFZHX1C|          5|           24|         25|   Y|     

In [14]:
# unpaid helpful reviews
unpaid_helpful_vine_df = helpful_vine_df.filter("vine == 'N'")
unpaid_helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [15]:
# Total number of reviews
reviews_total = helpful_vine_df.count()
reviews_total

39080

In [18]:
# Paid total 
paid_total = paid_helpful_vine_df.count()
paid_total

266

In [17]:
#unpaid total
unpaid_total = unpaid_helpful_vine_df.count()
unpaid_total

38814

In [21]:
#total five stars
total_five_stars = helpful_vine_df.filter("star_rating == 5")
count_total_five_stars = total_five_stars.count()
count_total_five_stars

18364

In [24]:
#total paid five start
paid_five_stars = paid_helpful_vine_df.filter("star_rating == 5")
count_paid_five_stars = paid_five_stars.count()
count_paid_five_stars

125

In [23]:
#total unpaid five start
unpaid_five_stars = unpaid_helpful_vine_df.filter("star_rating == 5")
count_unpaid_five_stars = unpaid_five_stars.count()
count_unpaid_five_stars

18239

In [32]:
#percent of five start reviews of total reviews 
percent_five_starts_total = (count_total_five_stars/reviews_total)*100
round(percent_five_starts_total, 2)

46.99

In [33]:
#percent of five star reviews od paid reviews 
percent_five_stars_paid = (count_paid_five_stars/paid_total)*100
round(percent_five_stars_paid, 2)

46.99

In [34]:
#Perent of five starts reviews unpaid reviews
percent_five_stars_unpaid = (count_unpaid_five_stars/unpaid_total)*100
round (percent_five_stars_unpaid, 2)

46.99